In [ ]:
# 导入库 Importing libraries
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import time
#导入Raspbot驱动库
#Import the Raspbot library
from Raspbot_Lib import Raspbot
# 创建Raspbot对象
# bot Create the Rosmaster object bot
bot = Raspbot()
import sys
sys.path.append('/home/pi/software/oled_yahboom/')
from yahboom_oled import *
# 创建oled对象 Create an oled object
oled = Yahboom_OLED(debug=False)
# 线程功能操作库 Thread function operation library
import threading
import inspect
import ctypes
import HSV_Config
#显示摄像头组件 Display camera components
image_widget = widgets.Image(format='jpeg', width=640, height=480)

In [ ]:
#bgr8转jpeg格式  bgr8 to jpeg format
import enum
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [ ]:
#USB摄像头打开方式 USB camera opening method

image = cv2.VideoCapture(0)  #打开摄像头/dev/video0 Open the camera /dev/video0
image.set(3, 320)
image.set(4, 240)
image.set(5, 30)  #设置帧率 Setting the frame rate
# image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 62) #设置亮度 -64 - 64  0.0 Set Brightness -64 - 64 0.0
# image.set(cv2.CAP_PROP_CONTRAST, 63) #设置对比度 -64 - 64  2.0 Set Contrast -64 - 64 2.0
# image.set(cv2.CAP_PROP_EXPOSURE, 4800) #设置曝光值 1.0 - 5000  156.0 Set the exposure value 1.0 - 5000 156.0

ret, frame = image.read()
image_widget.value = bgr8_to_jpeg(frame)
update_hsv = HSV_Config.update_hsv()


#CSI摄像头打开方式 CSI camera opening method
# from picamera2 import Picamera2, Preview
# import libcamera

# picam2 = Picamera2()  
# camera_config = picam2.create_preview_configuration(main={"format":'RGB888',"size":(640,480)})
# camera_config["transform"] = libcamera.Transform(hflip=1, vflip=1)
# picam2.configure(camera_config) 
# picam2.start() 
# frame = picam2.capture_array()
# image_widget.value = bgr8_to_jpeg(frame)
# update_hsv = HSV_Config.update_hsv()

### 颜色识别点亮RGB灯

In [ ]:
# 颜色到灯珠颜色编号的映射
color_to_number = {
        'red': 0,
        'green': 1,
        'blue': 2,
        'yellow': 3
    }
def light_2leds(color1,color2):
    # 获取颜色对应的灯珠颜色编号
    color1_number = color_to_number.get(color1, None)
    color2_number = color_to_number.get(color2, None)

    # 分配灯珠
    for i in range(1, 6):  # 前 5 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color1_number)
    for i in range(6, 11):  # 后 5 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color2_number)

def light_3leds(color1,color2,color3):
    # 获取颜色对应的灯珠颜色编号
    color1_number = color_to_number.get(color1, None)
    color2_number = color_to_number.get(color2, None)
    color3_number = color_to_number.get(color3, None)

    # 分配灯珠
    for i in range(1, 4):  # 前 3 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color1_number)
    for i in range(4, 8):  # 中 4 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color2_number)
    for i in range(8, 11):  # 后 3 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color3_number)

def light_4leds(color1,color2,color3,color4):
    # 获取颜色对应的灯珠颜色编号
    color1_number = color_to_number.get(color1, None)
    color2_number = color_to_number.get(color2, None)
    color3_number = color_to_number.get(color3, None)
    color4_number = color_to_number.get(color4, None)

    # 分配灯珠
    for i in range(1, 4):  # 前 3 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color1_number)
    for i in range(4, 6):  # 后 2 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color2_number)
    for i in range(6, 8):  # 后 2 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color3_number)
    for i in range(9, 11):  # 后 3 个灯珠
        bot.Ctrl_WQ2812_Alone(i, 1, color4_number)
    
def light_leds(unique_colors):
    if(len(unique_colors)==0):
        bot.Ctrl_WQ2812_ALL(0,0)
    elif(len(unique_colors)==1):
        bot.Ctrl_WQ2812_ALL(1,color_to_number.get(*unique_colors, None))
    elif(len(unique_colors)==2):
        light_2leds(*unique_colors)
    elif(len(unique_colors)==3):   
        light_3leds(*unique_colors)
    elif(len(unique_colors)==4):   
        light_4leds(*unique_colors)


### HSV值存储

In [ ]:
color_hsv  = {"red"   : ((0, 70, 72), (7, 255, 255)),
              "green" : ((54, 109, 78), (77, 255, 255)),
              "blue"  : ((92, 100, 62), (121, 251, 255)),
              "yellow": ((26, 100, 91), (32, 255, 255))}

In [ ]:
def TEST():
    detected_colors = []  # 用于存储检测到的颜色名称
    unique_colorstemp=None
    oled.init_oled_process() #初始化oled进程 Initialize oled process
    global color_hsv
    display(image_widget)
    while True:
        ret, frame = image.read() #usb摄像头 usb camera
        # frame = picam2.capture_array() #CSI摄像头 CSI Camera
        frame, binary,hsvname=update_hsv.get_contours(frame,color_hsv)
        unique_colors = list(dict.fromkeys(hsvname))
        # 根据列表的长度来决定如何分割字符串
        num_colors = len(unique_colors)
        # 如果列表中有4个元素
        if num_colors == 4:
            first_line = ', '.join(unique_colors[:2])
            second_line = ', '.join(unique_colors[2:])
        elif num_colors == 3:
            first_line = ', '.join(unique_colors[:2])
            second_line = unique_colors[2]
        elif num_colors == 2:
            first_line = ', '.join(unique_colors)
            second_line = ""
        elif num_colors == 1:
            first_line = unique_colors[0]
            second_line = ""
        else:
            first_line = ""
            second_line = ""
        if(unique_colors!=unique_colorstemp):
            light_leds(unique_colors)
            unique_colorstemp=unique_colors.copy()
            oled.clear()
            oled.add_line("color_detection:", 1)
            # 打印到OLED屏幕上
            oled.add_line(first_line, 2)
            oled.add_line(second_line, 3)
            oled.refresh()

        # 检查是否有足够的颜色框被检测到
        #cv2.putText(frame, detected_colors, 100, 200,
                           #cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
        
        # 实时传回图像数据进行显示 Real-time image data transmission for display
        image_widget.value = bgr8_to_jpeg(frame)
        

In [ ]:
thread1 = threading.Thread(target=TEST)
thread1.daemon = True
thread1.start()

In [ ]:
stop_thread(thread1)

In [ ]:
#usb摄像头 usb camera
#使用完成对象记住释放掉对象，不然下一个程序使用这个对象模块会被占用，导致无法使用
#After using the object, remember to release the object, otherwise the object module will be occupied by the next program, making it unusable
image.release()
bot.Ctrl_WQ2812_ALL(0,0)
# 恢复屏幕基础数据显示 Restore basic data display on screen
os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")
del bot
#CSI摄像头 CSI Camera
# picam2.stop()
# picam2.close()